# Using Evolution on Cyclic Graphs

Cycles are a simple graph with a known competitive coloring number. Both even and odd cycles have competitive coloring numbers of 3. We know that it is impossible for a cycle (of order > 2) to have a competitive coloring number of 2. Take, for example, a cycle of order 3. Each node has two neighbors, so coloring node 0 with color 1 means that nodes 1 and 2 need colors other than 1, but they cannot share the same color as they are connected. Thus at least three colors are required.

## Setup

As always, we begin with our imports.

In [1]:
import sys
sys.path.append("../..")

from classes.genetic_algorithm import GeneticAlgorithm
from graph_coloring.classes.gc_ruleset import GCRuleset
from graph_coloring.classes.gc_random_init_strategy import GCRandomInitStrategy

To begin, we will define a function that generates a cycle on a provided number of vertices.

In [2]:
def generate_cycle(num_vertices):
    """
    Generates a cycle with the provided number of vertices.
    
    Parameters:
        num_vertices (int): Number of vertices to generate for this cycle.
        
    Returns:
        A cycle on the provided number of vertices.
    """
    cycle = []
    for vtx in range(num_vertices):
        # Set previous vertex (vtx - 1) with the special case for vertex 0
        prev_vtx = num_vertices - 1 if vtx == 0 else vtx - 1
        
        # Set the next vertex (vtx + 1) and account for the special case of the last vertex
        next_vtx = (vtx + 1) % num_vertices
        
        # Append the node
        cycle.append({"color": 0, "adj": [prev_vtx, next_vtx]})
    
    return cycle

Next, we need to create a new `GeneticAlgorithm` object. Note that the constructor for this object takes a multitude of parameters by name, so let's format it nicely for future changes.

Most of the parameters, like population size and fitness, are optional. The only mandatory parameters are `ruleset`, `random_on_init_strategy`, and `strat_data`. The former is the ruleset for the game being tested. The latter two define how each member of the population receives a random strategy to play with.

The only two aspects of the genetic algorithm we are concerned with at this time are the number of nodes in the cycle and the number of colors in the game, so let us define a function to streamline this process for the rest of this notebook.

In [20]:
def cycle_test(num_vertices, num_colors):
    """
    Runs an evolutionary algorithm on a cycle graph with the provided number of vertices and colors.
    
    Parameters:
        num_vertices (int): Order of the cycle.
        num_colors (int): Number of colors for this game.
        
    Returns:
        Two populations of players, after evolution.
    """
    # Define a cycle to play on
    initial_state = generate_cycle(num_vertices)

    # Ruleset of the game being played on
    ruleset = GCRuleset("Graph Coloring Ruleset", initial_state, bounds = num_colors)

    # Create a new Evolution instance with the example strategy
    gen_algo = GeneticAlgorithm(
        ruleset,
        # Random-on-initialization strategy for generating populations of random players
        random_on_init_strat = GCRandomInitStrategy,
        # Data to be used by the above strategy
        strat_data = {"vertices": range(len(ruleset.initial_state)), "colors": range(1, ruleset.bounds + 1)},
        # Size of the populations
        pop_size = 100,
        # Number of generations to iterate through
        iterations = 10,
        # Minimum number of games each player must play during a generation
        num_games = 10,
        # Starting fitness threshold
        fitness = 0.5,
        # Maximum fitness threshold; be careful of setting this too close to 1.0
        max_fitness = 0.9,
        # How much the fitness threshold should increment after each iteration
        fitness_increment = 0.025,
        # Chance of a mutation to occur during player reproduction
        mutation_rate = 0.025
    )

    p1_pop, p2_pop = gen_algo.evolve(to_df=True)
    
    return p1_pop, p2_pop

## Even Cycles

Now we simply call our function. We will begin with a cycle of order 6 and 3 colors.

In [22]:
p1_pop, p2_pop = cycle_test(num_vertices=6, num_colors=3)

Once our populations have finished evolving, we can view each of them to see the top players in each.

In [23]:
p1_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
2,Player 1,0,"[0, 2, 5, 1, 3, 4]","[3, 2, 1]",1.0,222,0
15,Player 1,0,"[2, 4, 1, 3, 0, 5]","[2, 1, 3]",1.0,220,0
34,Player 1,0,"[2, 1, 5, 0, 4, 3]","[2, 3, 1]",1.0,220,0
22,Player 1,0,"[1, 2, 4, 0, 5, 3]","[2, 3, 1]",1.0,218,0
67,Player 1,0,"[1, 3, 4, 0, 5, 2]","[1, 3, 2]",1.0,217,0


In [24]:
p2_pop.head()

,Name,Gen,Vertices,Colors,Fitness,Wins,Losses
0,Player 2,0,"[3, 0, 1, 2, 4, 5]","[1, 2, 3]",0.0,0,0
1,Player 2,0,"[4, 0, 2, 1, 3, 5]","[2, 3, 1]",0.0,0,0
2,Player 2,0,"[3, 2, 4, 5, 0, 1]","[2, 3, 1]",0.0,0,0
3,Player 2,0,"[2, 4, 3, 0, 1, 5]","[3, 1, 2]",0.0,0,0
4,Player 2,0,"[3, 1, 2, 0, 4, 5]","[2, 3, 1]",0.0,0,0


It appears that with 3 colors, player 1 can be guaranteed to win 100% of the time on a cycle of order 6. Let us test this again on a graph of 8 vertices.

In [27]:
p1_pop, p2_pop = cycle_test(num_vertices=8, num_colors=3)
print(p1_pop.head())
print(p2_pop.head())

        Name Gen                  Vertices     Colors  Fitness Wins Losses
80  Player 1   0  [4, 6, 1, 7, 3, 2, 0, 5]  [3, 1, 2]      1.0  222      0
24  Player 1   0  [2, 3, 1, 4, 6, 5, 0, 7]  [1, 3, 2]      1.0  219      0
84  Player 1   0  [5, 2, 4, 3, 1, 6, 0, 7]  [1, 3, 2]      1.0  219      0
17  Player 1   0  [6, 4, 7, 5, 1, 3, 0, 2]  [1, 2, 3]      1.0  218      0
83  Player 1   0  [3, 1, 5, 4, 0, 6, 7, 2]  [2, 3, 1]      1.0  218      0
       Name Gen                  Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [1, 4, 7, 3, 2, 5, 0, 6]  [1, 2, 3]      0.0    0      0
1  Player 2   0  [6, 0, 4, 3, 5, 1, 2, 7]  [2, 1, 3]      0.0    0      0
2  Player 2   0  [6, 7, 4, 0, 3, 2, 5, 1]  [2, 3, 1]      0.0    0      0
3  Player 2   0  [1, 4, 6, 2, 7, 3, 5, 0]  [1, 3, 2]      0.0    0      0
4  Player 2   0  [5, 3, 7, 1, 2, 6, 4, 0]  [2, 1, 3]      0.0    0      0


Again we see 100% victory in the player 1 population. Let us try on some arbitrary cycle sizes.

In [31]:
for order in [100, 412, 876, 1492, 10574]:
    print("\nTesting on cycle of order " + str(order))
    p1_pop, p2_pop = cycle_test(num_vertices=8, num_colors=3)
    print(p1_pop.head())
    print(p2_pop.head())


Testing on cycle of order 100
        Name Gen                  Vertices     Colors  Fitness Wins Losses
57  Player 1   0  [5, 7, 2, 6, 4, 0, 3, 1]  [2, 3, 1]      1.0  219      0
37  Player 1   0  [7, 6, 3, 2, 0, 4, 1, 5]  [2, 3, 1]      1.0  218      0
90  Player 1   0  [1, 4, 0, 6, 5, 7, 3, 2]  [3, 2, 1]      1.0  217      0
24  Player 1   0  [6, 3, 7, 5, 1, 0, 2, 4]  [3, 1, 2]      1.0  216      0
46  Player 1   0  [0, 5, 2, 6, 4, 7, 1, 3]  [3, 2, 1]      1.0  216      0
       Name Gen                  Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [2, 3, 6, 1, 0, 7, 5, 4]  [1, 3, 2]      0.0    0      0
1  Player 2   0  [1, 5, 4, 0, 7, 6, 2, 3]  [2, 1, 3]      0.0    0      0
2  Player 2   0  [4, 2, 7, 6, 3, 0, 5, 1]  [2, 1, 3]      0.0    0      0
3  Player 2   0  [7, 0, 4, 5, 2, 1, 6, 3]  [3, 2, 1]      0.0    0      0
4  Player 2   0  [6, 1, 0, 3, 4, 5, 7, 2]  [1, 2, 3]      0.0    0      0

Testing on cycle of order 412
        Name Gen                  Vertices  

While this is not as thorough as a mathematical proof, it is clear that player 1 can have a 100% success rate on an even cycle with 3 colors. Now we will focus our attention to odd cycles.

## Odd Cycles

In [34]:
p1_pop, p2_pop = cycle_test(num_vertices=9, num_colors=3)
print(p1_pop.head())
print(p2_pop.head())

        Name Gen                     Vertices     Colors  Fitness Wins Losses
20  Player 1   0  [5, 1, 2, 8, 0, 3, 4, 7, 6]  [3, 2, 1]      1.0  224      0
57  Player 1   0  [1, 8, 2, 4, 5, 7, 0, 6, 3]  [2, 1, 3]      1.0  223      0
68  Player 1   0  [1, 5, 4, 8, 2, 3, 6, 7, 0]  [1, 2, 3]      1.0  221      0
99  Player 1   0  [0, 8, 5, 1, 3, 6, 4, 7, 2]  [3, 1, 2]      1.0  221      0
82  Player 1   0  [8, 2, 7, 6, 0, 4, 5, 3, 1]  [3, 1, 2]      1.0  219      0
       Name Gen                     Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [7, 6, 2, 4, 3, 1, 0, 8, 5]  [1, 2, 3]      0.0    0      0
1  Player 2   0  [0, 4, 2, 1, 3, 6, 7, 5, 8]  [1, 3, 2]      0.0    0      0
2  Player 2   0  [6, 7, 4, 2, 3, 5, 1, 8, 0]  [2, 1, 3]      0.0    0      0
3  Player 2   0  [5, 3, 2, 0, 1, 6, 8, 4, 7]  [1, 3, 2]      0.0    0      0
4  Player 2   0  [3, 7, 1, 2, 8, 4, 0, 5, 6]  [2, 1, 3]      0.0    0      0


Again we see expected behavior. On an odd cycle, player 1 is winning 100% of the time with 3 colors. Let us test this again on a variety of random graphs.

In [35]:
for order in [103, 415, 879, 1495, 10571]:
    print("\nTesting on cycle of order " + str(order))
    p1_pop, p2_pop = cycle_test(num_vertices=8, num_colors=3)
    print(p1_pop.head())
    print(p2_pop.head())


Testing on cycle of order 103
        Name Gen                  Vertices     Colors  Fitness Wins Losses
20  Player 1   0  [4, 7, 5, 0, 1, 3, 6, 2]  [1, 3, 2]      1.0  226      0
27  Player 1   0  [4, 5, 7, 2, 6, 1, 3, 0]  [3, 2, 1]      1.0  222      0
19  Player 1   0  [2, 1, 7, 5, 0, 6, 3, 4]  [1, 3, 2]      1.0  219      0
97  Player 1   0  [7, 4, 2, 6, 1, 3, 5, 0]  [3, 2, 1]      1.0  217      0
52  Player 1   0  [1, 6, 5, 2, 7, 3, 4, 0]  [1, 2, 3]      1.0  215      0
       Name Gen                  Vertices     Colors  Fitness Wins Losses
0  Player 2   0  [1, 4, 5, 3, 0, 7, 2, 6]  [3, 1, 2]      0.0    0      0
1  Player 2   0  [0, 6, 3, 7, 5, 1, 4, 2]  [3, 2, 1]      0.0    0      0
2  Player 2   0  [0, 7, 5, 6, 4, 2, 1, 3]  [2, 1, 3]      0.0    0      0
3  Player 2   0  [2, 1, 0, 3, 5, 7, 4, 6]  [2, 1, 3]      0.0    0      0
4  Player 2   0  [0, 1, 5, 2, 7, 6, 4, 3]  [3, 1, 2]      0.0    0      0

Testing on cycle of order 415
        Name Gen                  Vertices  

As expected, we see a clear 100% win rate for player 1 on odd cycles. This is, again, an expected result.

## Conclusion

From the experiments listed above, it is clear that player 1 wins 100% of the time on various cycles. It is known that cycles have a competitive coloring number of 3, and our results support this.